# Handling SC2017 PRP productions

This module has functions to:
- create all the workload
- move from state to state [created,training,trained,generating,generated,transporting,done]


In [3]:
from time import time
from elasticsearch import Elasticsearch, exceptions as es_exceptions
es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

 [--nb-epochs NB_EPOCHS] 
 [--batch-size BATCH_SIZE]
 [--latent-size LATENT_SIZE] 
 [--disc-lr DISC_LR]
 [--gen-lr GEN_LR] 
 [--adam-beta ADAM_BETA] 
 [--prog-bar]
 [--no-attn] 
 [--debug] 
 [--d-pfx D_PFX] 
 [--g-pfx G_PFX]
 dataset

In [6]:
def create_workload():
    id=0
    l1 = ['--disc-lr=0.0001','--disc-lr=0.0002','--disc-lr=0.0005']
    l2  = ['--gen-lr=0.00001','--gen-lr=0.00002','--gen-lr=0.00005']
    l3    = ['', '--no-attn']
    for a in l1:
        for b in l2:
            for c in l3:
                doc={}
                doc['created']=time()
                doc['status']='created'
                doc['training_options']=['--nb-epochs=100', a, b, c]
                doc['generating_options']=['--input_folder=XXX', '--output_folder=YYY', '--sets=ZZZ', '--showers=100000']
                id+=1
                print(doc)
                es.create(index='sc2017', doc_type='doc', id=id, body=doc)
create_workload()

{'created': 1509815918.299336, 'generating_options': ['--input_folder=XXX', '--output_folder=YYY', '--sets=ZZZ', '--showers=100000'], 'training_options': ['--nb-epochs=100', '--disc-lr=0.0001', '--gen-lr=0.00001', ''], 'status': 'created'}
{'created': 1509815922.9227629, 'generating_options': ['--input_folder=XXX', '--output_folder=YYY', '--sets=ZZZ', '--showers=100000'], 'training_options': ['--nb-epochs=100', '--disc-lr=0.0001', '--gen-lr=0.00001', '--no-attn'], 'status': 'created'}
{'created': 1509815922.9888306, 'generating_options': ['--input_folder=XXX', '--output_folder=YYY', '--sets=ZZZ', '--showers=100000'], 'training_options': ['--nb-epochs=100', '--disc-lr=0.0001', '--gen-lr=0.00002', ''], 'status': 'created'}
{'created': 1509815923.054772, 'generating_options': ['--input_folder=XXX', '--output_folder=YYY', '--sets=ZZZ', '--showers=100000'], 'training_options': ['--nb-epochs=100', '--disc-lr=0.0001', '--gen-lr=0.00002', '--no-attn'], 'status': 'created'}
{'created': 15098159